<a href="https://colab.research.google.com/github/DCorradetti/images/blob/main/AACA_SpinH3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install sympy galgebra

In [8]:
import sympy
from sympy import symbols
from galgebra.ga import Ga
from galgebra.printer import latex
from IPython.display import Math
import numpy
import math
import random

# tell sympy to use our printing by default
sympy.init_printing(latex_printer=latex, use_latex='mathjax')

s3coords = (x,y,z) = symbols('1 2 3', real=True)
s3 = Ga('e',
         g=[1,1,1],
         coords=s3coords)

M = s3.mv('M','mv',f = True)


e1, e2, e3 = s3.mv()

In [9]:
import copy
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt

def closure_of_generators(gens):
    elts = copy.copy(gens)
    new = copy.copy(gens)

    while True:
        new1 = [ ]
        for u in new:
            for v in gens:
                tmp = u * v
                if not tmp in elts:
                    if not tmp in new1:
                        new1.append(tmp)
        if new1 == [ ]:
            break
        else:
            print(len(elts),len(new1))
            print('----------------')
            elts = elts+new1
            new = copy.copy(new1)
    print('There are ', len(elts), ' elements in total.')
    return elts

def closure_of_roots(simples):
    simple_roots = [a for a in simples]
    roots = [a for a in simples]
    new = [a for a in simples]
    while 1==1:
        new1 = [ ]
        for u in new:
            for n in simple_roots:
                tmp = -n * u * n
                if not tmp in roots:
                    if not tmp in new1:
                        new1.append(tmp)
        if new1 == [ ]:
            break
        else:
            print(len(roots),len(new1))
            print('----------------')
            roots = roots+new1
            new = [a for a in new1]
    print('This is a root system with ', len(roots), ' roots')
    return roots

def order_of_element(m):
    tmp = m
    count = 1
    id = e1*e1
    while tmp != id:
        tmp = tmp*m
        count += 1
    return count

def find_inverses(group):
    g_inv = []
    m = group[0]
    id = e1*e1
    for i in group:
        for j in group:
            if i*j == id:
                g_inv.append(j)
                break
    return g_inv


def conjugacy_classes(elts):

    elts_inv = find_inverses(elts)
    classes = []
    leftover = set(range(0,len(elts)))

    for i in range(len(elts)):
        if i in leftover:
            newclass  = set([i])
            for j in range(len(elts)):
                if j != i:
                    tmp = elts[j]*elts[i]*elts_inv[j]
                    for k in range(len(elts)):
                        if k in leftover:
                            if tmp == elts[k]:
                                newclass.add(k)
                                leftover = leftover.difference([k])
                                break
            classes.append(newclass)
            print(len(leftover))
    return classes


def spIP(R1, R2):

    ip = 1/2*(R1*R2.rev()+R2*R1.rev())

    return ip

def CM(simple_roots_4D):

    A = zeros(len(simple_roots_4D), len(simple_roots_4D))
    for i in range(len(simple_roots_4D)):
        for j in range(len(simple_roots_4D)):
            tmp = spIP(simple_roots_4D[i], simple_roots_4D[j])
            A[i,j]=2*tmp.blade_coefs()[0]
    return A

In [10]:
def find_ito_generators_recursive_pin(gens, given_element):
    done = False
    for g in range(len(gens)):
        if gens[g] == given_element:
            print('Done: generator', gens[g] , 'Generator ', g)
            done = True
            return gens[g]


    elts = copy.copy(gens)
    new = copy.copy(gens)
    count = 0
    while done == False:
        new1 = [ ]
        for u in new:
            for v in range(len(gens)):
                tmp = u * gens[v]
                if not tmp in elts:
                    if not tmp in new1:
                        new1.append(tmp)
                        count+=1
                        #print(tmp, v)
                        if given_element == tmp:
                            print('-------')
                            print('found it',  gens[v], 'Generator ',v)
                            print('-------')
                            find_ito_generators_recursive(gens, u)
                            done = True
                            break

        if new1 == [ ]:
            break
        else:
            print(len(elts),len(new1))
            print('----------------')
            elts = elts+new1
            new = copy.copy(new1)
    print('There are ', len(elts), ' elements in total.')
    return

In [11]:
def closure_of_generators_words(gens):
    elts = copy.copy(gens)
    new = copy.copy(gens)
    path = []
    for i in range(len(gens)):
        path.append(str(i+1))

    while True:
        new1 = [ ]
        for j in range(len(new)):
            for k in range(len(gens)):
                tmp = new[j] * gens[k]
                if not tmp in elts:
                    if not tmp in new1:
                        new1.append(tmp)
                        path.append((path[len(elts)-len(new)+j])+str(k+1))
                        print(tmp, 'generator path is ',(path[len(elts)-len(new)+j])+str(k+1))
                        #print(len(elts)+len(new1), len(path), j)
        if new1 == [ ]:
            break
        else:
            print(len(elts),len(new1))
            print('----------------')
            elts = elts+new1
            new = copy.copy(new1)
    print('There are ', len(elts), ' elements in total.')
    return elts, path

In [12]:
def fourDvecto3Dspinor(vector):
    a0 = vector[0]
    a3 = vector[1]
    a2 =-vector[2]
    a1 = vector[3]
    B =a0+a1*(e2^e3) - a2*(e1^e3) + a3*(e1^e2)
    return B

def reflect_spinor(R1, R2):
    B = -R1*R2.rev()*R1
    return B

def closure_of_roots_in_spin(simples):
    simple_roots = [a for a in simples]
    roots = [a for a in simples]
    new = [a for a in simples]
    while 1==1:
        new1 = [ ]
        for u in new:
            for n in simple_roots:
                tmp = reflect_spinor(n, u)
                if not tmp in roots:
                    if not tmp in new1:
                        new1.append(tmp)
        if new1 == [ ]:
            break
        else:
            print(len(roots),len(new1))
            print('----------------')
            roots = roots+new1
            new = [a for a in new1]
    print('This is a root system with ', len(roots), ' roots')
    return roots

def closure_of_roots_in_spin_limit(simples, limit):
    simple_roots = [a for a in simples]
    roots = [a for a in simples]
    new = [a for a in simples]
    while 1==1:
        new1 = [ ]
        for u in new:
            for n in simple_roots:
                tmp = reflect_spinor(n, u)
                if not tmp in roots:
                    if not tmp in new1:
                        new1.append(tmp)
        if new1 == [ ]:
            break
        else:
            print(len(roots),len(new1))
            print('----------------')
            roots = roots+new1
            new = [a for a in new1]
            if len(roots)>limit:
                break
    print('This is a root system with ', len(roots), ' roots')
    return roots

In [13]:
def conjugacy_classes_quicker(elts):

    elts_inv = find_inverses_quicker(elts)
    print('inverses found')
    classes = []
    leftover = set(range(0,len(elts)))

    while len(leftover)>0:
        i = random.choice(tuple(leftover))
        newclass  = set([i])
        for j in range(len(elts)):
            if j != i:
                tmp = elts[j]*elts[i]*elts_inv[j]
                for k in range(len(elts)):
                    if k in leftover:
                        if tmp == elts[k]:
                            newclass.add(k)
                            leftover = leftover.difference([k])
                            break
        classes.append(newclass)
        print(len(leftover))
    return classes

def find_inverses_quicker(group):
    g_inv = []
    id = e1*e1
    for i in group:
        j = i.rev()
        if i*j == id:
            g_inv.append(j)
        elif i*j == -id:
            g_inv.append(-j)
        else:
            print('theres a problem')
    return g_inv

# Setting up the $H_3$ simple roots and computing the pin and spin group in terms of words in the generators/simple roots

In [14]:
tau=1/2*(1+sympy.sqrt(5))

a1, a2, a3 = e2, 1/2*(-tau*e1-e2-(tau-1)*e3), e1

Pin_H3_elts, Pin_H3_paths = closure_of_generators_words([a1, a2, a3])

1 generator path is  11
-0.5 + (0.25 + 0.25*sqrt(5))*e_1^e_2 + (0.25 - 0.25*sqrt(5))*e_2^e_3 generator path is  12
-e_1^e_2 generator path is  13
-0.5 + (-0.25*sqrt(5) - 0.25)*e_1^e_2 + (-0.25 + 0.25*sqrt(5))*e_2^e_3 generator path is  21
-0.25*sqrt(5) - 0.25 + 0.5*e_1^e_2 + (-0.25 + 0.25*sqrt(5))*e_1^e_3 generator path is  23
e_1^e_2 generator path is  31
-0.25*sqrt(5) - 0.25 - 0.5*e_1^e_2 + (0.25 - 0.25*sqrt(5))*e_1^e_3 generator path is  32
3 7
----------------
(0.25 + 0.25*sqrt(5))*e_1 - 0.5*e_2 + (-0.25 + 0.25*sqrt(5))*e_3 generator path is  121
-0.5*e_1 + (-0.25*sqrt(5) - 0.25)*e_2 + (0.25 - 0.25*sqrt(5))*e_1^e_2^e_3 generator path is  123
-e_1 generator path is  131
0.5*e_1 + (-0.25*sqrt(5) - 0.25)*e_2 + (-0.25 + 0.25*sqrt(5))*e_1^e_2^e_3 generator path is  132
-0.5*e_1 + (0.25 + 0.25*sqrt(5))*e_2 + (-0.25 + 0.25*sqrt(5))*e_1^e_2^e_3 generator path is  213
0.5*e_1 + (-0.25*sqrt(5) - 0.25)*e_2 + (0.25 - 0.25*sqrt(5))*e_1^e_2^e_3 generator path is  231
(-0.25 + 0.25*sqrt(5))*e_1 +

# Calculate conjugacy classes

In [15]:
pin_CC_H3 = conjugacy_classes_quicker(Pin_H3_elts)

inverses found
220
208
196


KeyboardInterrupt: ignored

In [ ]:
for cclass in pin_CC_H3:
    print(  '\n  \hline \n')
    for element in sorted(list(cclass)):
        #print('$', order_of_element(Pin_H3_elts[element]),'$', '& ', '$',element+1,'$', '& $',  latex(2*Pin_H3_elts[element]), ' $ &', Pin_H3_paths[element], '\\\\')
        print(order_of_element(Pin_H3_elts[element]),element+1,(2*Pin_H3_elts[element]),  Pin_H3_paths[element])


# Grand antiprism and snub 24-cell

Use subrootsystems to split the 600-cell into two separately invariant polytopes. Explicitly verify they are stabilised.

In [16]:
spin_group_H3 = closure_of_generators([a1*a2, a2*a3])

2 4
----------------
6 8
----------------
14 12
----------------
26 15
----------------
41 18
----------------
59 19
----------------
78 19
----------------
97 15
----------------
112 8
----------------
There are  120  elements in total.


In [17]:
def closed_under_spin_reflections(points, group):
    for u in points:
        for n in group:
            tmp = reflect_spinor(n, u)
            if not tmp in points:
                print('issue')
                return False
                break
    return True

$A_2\oplus A_2$ example

In [18]:
A2A2 = closure_of_roots_in_spin([a1*a1, a1*a2, e1*e2*e3*a1, e1*e2*e3*a2])

4 6
----------------
10 2
----------------
This is a root system with  12  roots


In [19]:
for dec in A2A2:
    print(dec in spin_group_H3)

True
True
True
True
True
True
True
True
True
True
True
True


In [20]:
compA2A2 = []

for elmt in spin_group_H3:
    if not elmt  in A2A2:
        compA2A2.append(elmt)
len(compA2A2)

108

In [21]:
A2A2stable = closed_under_spin_reflections(A2A2, A2A2)
A2A2stable

True

In [22]:
compA2A2stable = closed_under_spin_reflections(compA2A2, A2A2)
compA2A2stable

True

$H_2\oplus H_2$ example (Grand Antiprism)

In [23]:
H2H2 = closure_of_roots_in_spin([a1*a1, a3*a2, e1*e2*e3*a3, e1*e2*e3*a2])

4 8
----------------
12 6
----------------
18 2
----------------
This is a root system with  20  roots


In [24]:
for dec in H2H2:
    print(dec in spin_group_H3)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [25]:
GA = []

for elmt in spin_group_H3:
    if not elmt  in H2H2:
        GA.append(elmt)
len(GA)

100

In [26]:
closed_under_spin_reflections(GA, H2H2)

True

In [27]:
H2H2stable = closed_under_spin_reflections(H2H2, H2H2)
H2H2stable

True

Aside for different choices of simple roots for the subrootsystems we need:

$H_2\oplus H_2$ (see separate worksheet for doubling)

In [ ]:
from sympy.matrices import Matrix, eye, zeros, ones, diag, GramSchmidt

simple_roots_4D = [a1*a1, a2*a3, a3*e1*e2*e3,  a2*e1*e2*e3]

A = zeros(4, 4)
for i in range(len(simple_roots_4D)):
    for j in range(len(simple_roots_4D)):
        tmp = spIP(simple_roots_4D[i], simple_roots_4D[j])
        A[i,j]=2*tmp.blade_coefs()[0]
A

$A_2\oplus A_2$

In [29]:
simple_roots_4D = [a1*a1, a1*a2, a2*e1*e2*e3,  a1*e1*e2*e3]

A = zeros(4, 4)
for i in range(len(simple_roots_4D)):
    for j in range(len(simple_roots_4D)):
        #print(simple_roots_4D[i],simple_roots_4D[j])
        tmp = spIP(simple_roots_4D[i], simple_roots_4D[j])
        #print(tmp.blade_coefs()[0])
        A[i,j]=2*tmp.blade_coefs()[0] #spIP(simple_roots_4D[i], simple_roots_4D[j])
A

Matrix([
[ 2.0, -1.0,    0,    0],
[-1.0,  2.0,    0,    0],
[   0,    0,  2.0, -1.0],
[   0,    0, -1.0,  2.0]])

$H_4$

In [30]:
simple_roots_4D = [a1*a1, a1*a2,  e1*e2*a2*e3, a2*e1*e2*e3]

A = CM(simple_roots_4D)
A


Matrix([
[ 2.0, -1.0,                  0,                  0],
[-1.0,  2.0,               -1.0,                  0],
[   0, -1.0,                2.0, -0.5*sqrt(5) - 0.5],
[   0,    0, -0.5*sqrt(5) - 0.5,                2.0]])

In [ ]:
H4_RS=closure_of_roots_in_spin(simple_roots_4D)

4 8
----------------
12 8
----------------
20 8
----------------
28 8
----------------
36 8
----------------
44 8
----------------
52 7
----------------
59 6
----------------


$A_4$

In [ ]:
CM([a1*a1, a1*a2, a1*a3*a2*a1*a3*a2*a1*a3, a3*a2*a1*a3*a2*a1*a3*a2*a3*a1*a2*a3])

In [ ]:
A4 = closure_of_roots_in_spin([a1*a1, a1*a2, a1*a3*a2*a1*a3*a2*a1*a3, a3*a2*a1*a3*a2*a1*a3*a2*a3*a1*a2*a3])

Carry on with the stability of the complementary polytope pairs here for the other cases whose simple roots we now have.

In [ ]:
A4snub = []

for elmt in spin_group_H3:
    if not elmt  in A4:
        A4snub.append(elmt)
len(A4snub)

In [ ]:
A4snub_stable = closed_under_spin_reflections(A4snub, A4)
A4snub_stable

$D_4$ and the snub 24-cell

In [ ]:
CM([a1*a1, a1*a2, a1*a2*a3*a2*a3*a1*a2*a3, a3*a2*a1*a3*a2*a1*a3*a2])

In [ ]:
D4 = closure_of_roots_in_spin([a1*a1, a1*a2, a1*a2*a3*a2*a3*a1*a2*a3, a3*a2*a1*a3*a2*a1*a3*a2])

In [ ]:
snub = []

for elmt in spin_group_H3:
    if not elmt  in D4:
        snub.append(elmt)
len(snub)

In [ ]:
D4stable = closed_under_spin_reflections(D4, D4)
D4stable

In [ ]:
snub_stable = closed_under_spin_reflections(snub, D4)
snub_stable

Just some routines to export the point lists

In [ ]:
def ThreeDspinortofourDvector(spinor):
    b0 = (spinor.blade_coefs()[0])
    b3 = (spinor.blade_coefs()[4])
    b2 =-(spinor.blade_coefs()[5])
    b1 = (spinor.blade_coefs()[6])
    B =[b0, b3, -b2, b1]
    return B

def ThreeDspinortofourDvector_num(spinor):
    b0 = (spinor.blade_coefs()[0])
    b3 = (spinor.blade_coefs()[4])
    b2 =-(spinor.blade_coefs()[5])
    b1 = (spinor.blade_coefs()[6])
    B =[round(float(b0), 3), round(float(b3), 3), round(float(-b2), 3), round(float(b1),3)]
    return B

def fourDvecto3Dspinor(vector):
    a0 = vector[0]
    a3 = vector[1]
    a2 =-vector[2]
    a1 = vector[3]
    B =a0+a1*(e2^e3) - a2*(e1^e3) + a3*(e1^e2)
    return B

$A_1^4$

In [ ]:
A14 = closure_of_roots_in_spin([e1*e1, e1*e2, e2*e3, e3*e1])

In [ ]:
snubA14 = []

for elmt in spin_group_H3:
    if not elmt  in A14:
        snubA14.append(elmt)
len(snubA14)

In [ ]:
A14stable = closed_under_spin_reflections(A14, A14)
A14stable

In [ ]:
A14stable = closed_under_spin_reflections(snubA14, A14)
A14stable

# Coxeter plane

Verify eigenvector and eigenvalue of the $H_4$ Cartan matrix

In [ ]:
C_H4 = CM(simple_roots_4D)
M_evec = zeros(4,1)
M_evec[0] = 4+4*sympy.sqrt(5)
M_evec[1] = 2*sympy.sqrt(7+sympy.sqrt(30+6*sympy.sqrt(5))+sympy.sqrt(5))*(1+sympy.sqrt(5))
M_evec[2] = sympy.sqrt(30+6*sympy.sqrt(5))*sympy.sqrt(5)+4*sympy.sqrt(5)+sympy.sqrt(30+6*sympy.sqrt(5))+8
M_evec[3] = (sympy.sqrt(5)+sympy.sqrt(30+6*sympy.sqrt(5))-1)*sympy.sqrt(7+sympy.sqrt(30+6*sympy.sqrt(5))+sympy.sqrt(5))

M_eval = 2-(1/2)*sympy.sqrt(7+sympy.sqrt(30+6*sympy.sqrt(5))+sympy.sqrt(5))

tmp = C_H4*M_evec-M_eval*M_evec
print(sympy.expand(tmp))

Find/verify the reciprocal basis of fundamental weights:

In [ ]:
reciprocal_roots = [1-tau*(e2^e3) - (tau+1)*(e1^e3), -2*tau*(e2^e3) - 2*(tau+1)*(e1^e3),-(2*tau+1)*(e2^e3) - 3*(tau+1)*(e1^e3) -tau*(e1^e2) ,-(2*tau+1)*(e2^e3) - (3*tau+1)*(e1^e3) -tau*(e1^e2)]

In [ ]:
Follow the 4D Coxeter plane discussion (e.g. from the Sage worksheet) entirely in spinorial terms in the even subalgebra. Rec = zeros(4)
for i in range(len(simple_roots_4D)):
    for j in range(len(reciprocal_roots)):
        tmp=((spIP(simple_roots_4D[i], reciprocal_roots[j])))
        Rec[i,j]=tmp.blade_coefs()[0]
Rec


Follow the 4D Coxeter plane discussion (e.g. from the Sage worksheet) entirely in spinorial terms in the even subalgebra.

In [ ]:
# find partition of roots
PF_vec = Maple_evec #PF_vec =vector([0.371748034514429893,0.707106781320503752,0.601500954816544975])#cheat

Omega_H4=reciprocal_roots

#could also define a function - do by hand for now!
roots_black_H4 = [0,2]
roots_white_H4 = [1,3]

black_vector_H4 = 0
white_vector_H4 = 0

for i in roots_black_H4:
    black_vector_H4 = black_vector_H4  +PF_vec[i]*Omega_H4[i]

for i in roots_white_H4:
    white_vector_H4 = white_vector_H4+PF_vec[i]*Omega_H4[i]

#these two vectors define the Coxeter plane. Now turn them into a Cartesian coordinate system there

#normalise
black_vector_H4 =black_vector_H4/(black_vector_H4.norm())
white_vector_H4 =white_vector_H4/(white_vector_H4.norm())


spIP(black_vector_H4, white_vector_H4)

#orthogonalise
black_vector_H4 = black_vector_H4 - spIP(black_vector_H4, white_vector_H4) *white_vector_H4
#renormalise
black_vector_H4 =black_vector_H4/(black_vector_H4.norm())

tmp = spIP(black_vector_H4, white_vector_H4)
float(tmp.blade_coefs()[0])

In [ ]:
float_black_vector_H4 =float(black_vector_H4.blade_coefs()[0])+float(black_vector_H4.blade_coefs()[6])*(e2^e3) +float(black_vector_H4.blade_coefs()[5])*(e1^e3) + float(black_vector_H4.blade_coefs()[4])*(e1^e2)

float_white_vector_H4 =float(white_vector_H4.blade_coefs()[0])+float(white_vector_H4.blade_coefs()[6])*(e2^e3) +float(white_vector_H4.blade_coefs()[5])*(e1^e3) + float(white_vector_H4.blade_coefs()[4])*(e1^e2)



Plotting here not as pretty as in Sage but in 2D and without the 3D bounding boxes.

In [ ]:
H4_RS_proj=[]
for i in H4_RS:
    tmp_b = spIP(i, float_black_vector_H4)
    tmp_w = spIP(i, float_white_vector_H4)
    H4_RS_proj.append([tmp_b.blade_coefs()[0], tmp_w.blade_coefs()[0]])

In [ ]:
import matplotlib.pyplot as plt

x = [i[0] for i in H4_RS_proj]
y = [i[1] for i in H4_RS_proj]
plt.axes().set_aspect('equal')
plt.xlabel('')
plt.ylabel('')
plt.axes().set_xticks([])
plt.axes().set_yticks([])
plt.scatter(x,y)

In [ ]:
A2A2_RS=A2A2
A2A2_RS_proj=[]
for i in A2A2_RS:
    tmp_b = spIP(i, float_black_vector_H4)
    tmp_w = spIP(i, float_white_vector_H4)
    A2A2_RS_proj.append([tmp_b.blade_coefs()[0], tmp_w.blade_coefs()[0]])
x = [i[0] for i in A2A2_RS_proj]
y = [i[1] for i in A2A2_RS_proj]
plt.axes().set_aspect('equal')
plt.xlabel('')
plt.ylabel('')
plt.axes().set_xticks([])
plt.axes().set_yticks([])
plt.scatter(x,y)

In [ ]:
H2H2_RS=H2H2
H2H2_RS_proj=[]
for i in H2H2_RS:
    tmp_b = spIP(i, float_black_vector_H4)
    tmp_w = spIP(i, float_white_vector_H4)
    H2H2_RS_proj.append([tmp_b.blade_coefs()[0], tmp_w.blade_coefs()[0]])
x = [i[0] for i in H2H2_RS_proj]
y = [i[1] for i in H2H2_RS_proj]
plt.axes().set_aspect('equal')
plt.xlabel('')
plt.ylabel('')
plt.axes().set_xticks([])
plt.axes().set_yticks([])
plt.scatter(x,y)

In [ ]:
A4_RS=A4
A4_RS_proj=[]
for i in A4_RS:
    tmp_b = spIP(i, float_black_vector_H4)
    tmp_w = spIP(i, float_white_vector_H4)
    A4_RS_proj.append([tmp_b.blade_coefs()[0], tmp_w.blade_coefs()[0]])
x = [i[0] for i in A4_RS_proj]
y = [i[1] for i in A4_RS_proj]
plt.axes().set_aspect('equal')
plt.xlabel('')
plt.ylabel('')
plt.axes().set_xticks([])
plt.axes().set_yticks([])
plt.scatter(x,y)

In [ ]:
D4_RS=D4
D4_RS_proj=[]
for i in D4_RS:
    tmp_b = spIP(i, float_black_vector_H4)
    tmp_w = spIP(i, float_white_vector_H4)
    D4_RS_proj.append([tmp_b.blade_coefs()[0], tmp_w.blade_coefs()[0]])
x = [i[0] for i in D4_RS_proj]
y = [i[1] for i in D4_RS_proj]
plt.axes().set_aspect('equal')
plt.xlabel('')
plt.ylabel('')
plt.axes().set_xticks([])
plt.axes().set_yticks([])
plt.scatter(x,y)

In [ ]:
A14_RS=A14
A14_RS_proj=[]
for i in A14_RS:
    tmp_b = spIP(i, float_black_vector_H4)
    tmp_w = spIP(i, float_white_vector_H4)
    A14_RS_proj.append([tmp_b.blade_coefs()[0], tmp_w.blade_coefs()[0]])
x = [i[0] for i in A14_RS_proj]
y = [i[1] for i in A14_RS_proj]
plt.axes().set_aspect('equal')
plt.xlabel('')
plt.ylabel('')
plt.axes().set_xticks([])
plt.axes().set_yticks([])
plt.scatter(x,y)

In [ ]:
A14_RS=snubA14
A14_RS_proj=[]
for i in A14_RS:
    tmp_b = spIP(i, float_black_vector_H4)
    tmp_w = spIP(i, float_white_vector_H4)
    A14_RS_proj.append([tmp_b.blade_coefs()[0], tmp_w.blade_coefs()[0]])
x = [i[0] for i in A14_RS_proj]
y = [i[1] for i in A14_RS_proj]
plt.axes().set_aspect('equal')
plt.xlabel('')
plt.ylabel('')
plt.axes().set_xticks([])
plt.axes().set_yticks([])
plt.scatter(x,y)

Final checks root systems are closed and contained in $H_4$.

In [ ]:

for i in [a1*e3,   a2*e1*e2*e3,a2*e3,a1*e1*e2*e3]:
    print(i in spin_group_H3)



In [ ]:
A2A2_spin_closure = closure_of_generators([a1*a1, a1*a2, e1*e2*e3*a1, e1*e2*e3*a2])


In [ ]:
H2H2_spin_closure = closure_of_generators([a1*a1, a2*a3, e1*e2*e3*a2, e1*e2*e3*a3])

In [ ]:
for i in A2A2_spin_closure:
    for j in A2A2_spin_closure:
        if i*j not in A2A2_spin_closure:
            print('problem')

In [ ]:
for i in H2H2_spin_closure:
    for j in H2H2_spin_closure:
        if i*j not in H2H2_spin_closure:
            print('problem')